# commands

> Commands are used to harness the power of nbdev.

In [ ]:
#| default_exp commands

In [ ]:
#| export
import types, pathlib, os
from functools import wraps
import typer
from typing_extensions import Annotated
from fastcore.docments import *
from fastcore.meta import delegates
from fastcore.script import call_parse
from fastcore.utils import *
from rich import print

In [ ]:
#| export
from nbdev import cli, release, config, quarto, doclinks, merge, migrate, sync
from nbdev import clean as nbclean
from nbdev import test as nbtest

In [ ]:
#| export
# remove callparse
nbdev_new = cli.nbdev_new.__wrapped__

@delegates(nbdev_new)
def new(
    target: Annotated[pathlib.Path, typer.Argument(help="Path to create project")],
    **kwargs):
    """
    Create an nbdev project. Examples:
    
    * In your current directory: `nbz new .`
    
    * In a different directory: `nbz new my-project`
    
    ---
    
    Learn more [nbz.answer.ai/commands#new](https://nbz.answer.ai/commands#new)
    """
    # kwargs['path'] = str(path)
    print(f'Changing directory to {target}')
    olddir = pathlib.Path('.')
    os.chdir(target)
    resp=nbdev_new(**kwargs)
    os.chdir(olddir)
    print(f'Changing directory back')
    return resp
new.rich_help_panel = 'Getting started'
new.no_args_is_help=False
